In [0]:
# Install all dependencies
!pip install pandas numpy tensorflow==2.1.0rc0 sklearn jupyterlab matplotlib pillow packaging

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.1.0-rc0


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls drive/My\ Drive/Colab\ Notebooks/data_tmp/dogs-vs-cats-small

test  train  valid


In [0]:
path_dir = "drive/My Drive/Colab Notebooks/data_tmp/dogs-vs-cats-small"

## Building the model

Combining the Vgg16 model with a single dense layer to train the model for this specific task

In [6]:
model_base = tf.keras.applications.vgg16.VGG16(
    weights = 'imagenet',
    include_top = False, # whether the densly connected layers also should be included
    input_shape = (150, 150, 3) # this actually doesn't need to be passed
)

58892288/58889256 [==============================] - 2s 0us/step


In [7]:
model_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
model = keras.Sequential([
    model_base,
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation = 'relu'),    
    keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
# Setting all Vgg16 layers as non-trainable
for layer in model_base.layers:
    layer.trainable = False

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 2e-5),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


## Data preprocessing

We're gonna use the data image generator to read data batches one by one from disck instead of loading them all into memory at once, with some data augmentation techniques to prevent ovefitting.

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255, 
    rotation_range = 40, # rotating the image randomly by a number of degrees to each of the side
    width_shift_range = 0.2, # sliding the image as a fraction of its width or height
    height_shift_range = 0.2, 
    shear_range = 0.2, # changing the angle of the image (shearing)
    zoom_range = 0.2, # slightly zooming inside the image
    horizontal_flip = True, # possible when there is no assumption of horizontal asymmetry
    fill_mode = 'nearest' # the strategy of how newly created pixels will be filled e.g. after width/ height shift
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255)

In [14]:
train_dir = path_dir + '/train'
valid_dir = path_dir + '/valid'

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (150, 150),
        batch_size = 20,
        class_mode = 'binary'
)

valid_generator = train_datagen.flow_from_directory(
        valid_dir,
        target_size = (150, 150),
        batch_size = 20,
        class_mode = 'binary'
)

Found 1999 images belonging to 2 classes.
Found 998 images belonging to 2 classes.


## Fitting the model

In [15]:
model.fit_generator(
    train_generator, 
    steps_per_epoch = 100, # 2000 samples, 20 per batches = 100 steps per each epoch
    epochs = 30,
    validation_data = valid_generator,
    validation_steps = 50 # 1000 samples, 20 per batch = 50 steps per each epoch
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/30
100/100 [==============================] - 851s 9s/step - loss: 0.6068 - accuracy: 0.6758 - val_loss: 0.5186 - val_accuracy: 0.7645
Epoch 2/30
100/100 [==============================] - 24s 245ms/step - loss: 0.4853 - accuracy: 0.7919 - val_loss: 0.4573 - val_accuracy: 0.7976
Epoch 3/30
100/100 [==============================] - 24s 242ms/step - loss: 0.4318 - accuracy: 0.8044 - val_loss: 0.4110 - val_accuracy: 0.8096
Epoch 4/30
100/100 [==============================] - 25s 248ms/step - loss: 0.3987 - accuracy: 0.8199 - val_loss: 0.3925 - val_accuracy: 0.8246
Epoch 5/30
100/100 [==============================] - 25s 247ms/step - loss: 0.3753 - accuracy: 0.8329 - val_loss: 0.3916 - val_accuracy: 0.8327
Epoch 6/30
100/100 [==============================] - 25s 245ms/step - loss: 0.3633 - accuracy: 0.8354 - val

In [16]:
# Saving the trained model
model.save('drive/My Drive/Colab Notebooks/data_tmp/model_cats_transfer_learning_v1', save_format = 'tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: drive/My Drive/Colab Notebooks/data_tmp/model_cats_transfer_learning_v1/assets


## Evaluate the model

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose = 2)

print('\nTest accuracy:', test_acc)

## Making predictions

In [0]:
predictions = model.predict(test_images)

In [0]:
predictions[0]

array([6.4292335e-22, 1.6219846e-22, 3.8224264e-22, 2.2197426e-20,
       1.6082298e-26, 4.0185693e-12, 2.1265178e-23, 2.1613789e-11,
       4.5142590e-21, 1.0000000e+00], dtype=float32)

In [0]:
np.argmax(predictions[0])

9